In [115]:
 import json

In [116]:
# Opening JSON file
f = open('contacts.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Closing file
f.close()


In [117]:
from typing import List

In [127]:
class Contact:
  def __init__(self, name, is_missing, possible_email_lists):
    self.is_empty: bool = False
    self.is_ambiguous: bool = False
    self._resolved_email: str = ""
    self._possible_emails: List[str] = []

    possible_email_lists = [email_list for email_list in possible_email_lists if len(email_list) > 0]

    self.name = name
    self._possible_email_lists = possible_email_lists

    for email_list in possible_email_lists:
      if not email_list:
        continue
      self._possible_emails.append(email_list[0])
    if is_missing:
      print(f"ERROR: {name} is missing email")
      self.is_empty = True
      return
    if len(possible_email_lists) == 1:
      self._resolved_email = possible_email_lists[0][0]
    elif len(possible_email_lists) > 1:
      try:
        self._resolved_email = self.disambiguate(possible_email_lists)
      except:
        print(f"ERROR: {name} has ambiguous emails: {possible_email_lists}")
        self.is_ambiguous = True


  def disambiguate(self, possible_email_lists):
    indices_with_cs_emails = set()
    indices_with_engineering_emails = set()

    for i, email_list in enumerate(possible_email_lists):
      if any("@cs." in email.lower() for email in email_list):
        indices_with_cs_emails.add(i)
      if any("@engineering." in email.lower() for email in email_list):
        indices_with_engineering_emails.add(i)

    print(f"{self.name} has {len(indices_with_cs_emails)} cs emails and {len(indices_with_engineering_emails)} engineering emails")

    if len(indices_with_cs_emails) == len(indices_with_engineering_emails) == 0:
      raise Exception("No CS or Engineering emails found")
    
    if (len(indices_with_cs_emails) > 1 or len(indices_with_engineering_emails) > 1):
      raise Exception("Ambiguous email")

    if (len(indices_with_engineering_emails) == 1):
      if (not indices_with_cs_emails.issubset(indices_with_cs_emails)):
        raise Exception("Ambiguous email")
      return possible_email_lists[indices_with_engineering_emails.pop()][0]
    if (len(indices_with_cs_emails) == 1):
      if (not indices_with_engineering_emails.issubset(indices_with_engineering_emails)):
        raise Exception("Ambiguous email")
      return possible_email_lists[indices_with_cs_emails.pop()][0]

  @property
  def resolved_email(self):
    if self.is_empty:
      raise Exception(f"{self.name} has no emails")
    if self.is_ambiguous:
      print(f"WARNING: {self.name} has ambiguous emails: {self._possible_email_lists}")
      return self._possible_emails
    return self._resolved_email


In [128]:
def from_json(contact_json):
  return Contact(
    contact_json["name"],
    contact_json["missing"],
    contact_json["possibilities"]
  )

In [129]:
contacts = []
for contact in data:
  contacts.append(from_json(contact))

ERROR: Chase Alcorn is missing email
Luis Barajas has 1 cs emails and 0 engineering emails
ERROR: Mac Beggs is missing email
ERROR: Sam Boersma is missing email
ERROR: Jessica Cai is missing email
Brian Chang has 1 cs emails and 0 engineering emails
Timothy Chang has 1 cs emails and 0 engineering emails
Jiakang Chen has 0 cs emails and 0 engineering emails
ERROR: Jiakang Chen has ambiguous emails: [['jiakang@ucsb.edu', 'jiakang@umail.ucsb.edu'], ['jchen758@ucsb.edu', 'jchen758@umail.ucsb.edu']]
ERROR: Alex Cho is missing email
ERROR: Dylan Chung is missing email
ERROR: Ishan Gondara is missing email
ERROR: Eric Han is missing email
ERROR: Kevin Heo is missing email
Brian Ho has 1 cs emails and 0 engineering emails
Allison Huang has 1 cs emails and 0 engineering emails
ERROR: Shawn Jiang is missing email
Julie Ku has 0 cs emails and 0 engineering emails
ERROR: Julie Ku has ambiguous emails: [['julieku@ucsb.edu', 'julieku@umail.ucsb.edu'], ['as-jku@ucsb.edu', 'jku@as.ucsb.edu']]
ERROR: K

In [130]:
emails = []
for contact in contacts:
  try:
    emails.append(contact.resolved_email)
  except Exception as e:
    print("ERROR:", e)


ERROR: Chase Alcorn has no emails
ERROR: Mac Beggs has no emails
ERROR: Sam Boersma has no emails
ERROR: Jessica Cai has no emails
ERROR: Alex Cho has no emails
ERROR: Dylan Chung has no emails
ERROR: Ishan Gondara has no emails
ERROR: Eric Han has no emails
ERROR: Kevin Heo has no emails
ERROR: Shawn Jiang has no emails
ERROR: Kevin Lai has no emails
ERROR: Sylvia Lin has no emails
ERROR: Ray Ling has no emails
ERROR: Amy Lyu has no emails
ERROR: Patrick Moon has no emails
ERROR: Zack Reardon has no emails
ERROR: Rowan Tran has no emails
ERROR: Yizhou Wu has no emails
ERROR: Shawn Xiao has no emails
ERROR: Carl Xiong has no emails
ERROR: Rex Xu has no emails
ERROR: Mena Yang has no emails
ERROR: Belle Zhang has no emails
ERROR: Belle Zhang has no emails


In [131]:
def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])


In [132]:
len(emails)

195

In [133]:
';'.join(flatten(emails))

'rithwik@ucsb.edu;safiyazalavi@ucsb.edu;arianalavi@ucsb.edu;kya@ucsb.edu;nirmit@ucsb.edu;m_atkinson@ucsb.edu;rohil@ucsb.edu;luisbarajas@ucsb.edu;corinnabasch@ucsb.edu;ebeltran@ucsb.edu;mpbonham@ucsb.edu;tufeicai@ucsb.edu;jcalpe@ucsb.edu;alexjcano@ucsb.edu;jcao@ucsb.edu;siyucao@ucsb.edu;acarbone@ucsb.edu;dylancervantes@ucsb.edu;justin_chang@ucsb.edu;bpc@ucsb.edu;timothychang@ucsb.edu;marvinchang@ucsb.edu;christopherchang@ucsb.edu;zhimei_chen@ucsb.edu;jiakang@ucsb.edu;jchen758@ucsb.edu;jiayuechen@ucsb.edu;melodychen@ucsb.edu;ychen861@ucsb.edu;kaiechen@ucsb.edu;sophiacheng@ucsb.edu;arik@ucsb.edu;irenecho@ucsb.edu;stevenchoi@ucsb.edu;dchou@umail.ucsb.edu;chiaanchou@ucsb.edu;yoonji@ucsb.edu;jaredconnolly@ucsb.edu;cbconrad@ucsb.edu;jcooley@umail.ucsb.edu;idelarosby@ucsb.edu;vdharasker@ucsb.edu;chloedinh-luong@ucsb.edu;jdysart@ucsb.edu;taraellingson@ucsb.edu;jenriquez@ucsb.edu;qiyuanfeng@ucsb.edu;jaredfitton@ucsb.edu;lawrencegao@ucsb.edu;longgao@ucsb.edu;wentao_gao@ucsb.edu;christiangarduno@u

In [134]:
ambiguous_contacts = [contact for contact in contacts if contact.is_ambiguous]

In [136]:
boring_domains = ["@umail.ucsb.edu", "@ucsb.edu", "@cs.ucsb.edu", "@engineering.ucsb.edu"]
for contact in ambiguous_contacts:
  if not all(any(domain in email for domain in boring_domains) for email in flatten(contact._possible_email_lists)):
    print(f"{contact.name} has interesting emails: {contact._possible_email_lists}")

Julie Ku has interesting emails: [['julieku@ucsb.edu', 'julieku@umail.ucsb.edu'], ['as-jku@ucsb.edu', 'jku@as.ucsb.edu']]
Regan Wheatley has interesting emails: [['as-rwheatley@ucsb.edu', 'rwheatley@as.ucsb.edu'], ['rwheatley@ucsb.edu', 'rwheatley@umail.ucsb.edu']]
